In [6]:
import pandas as pd
import numpy as np
from typing import List, Tuple

class GlassPiece:
    def __init__(self, location: str, length: float, height: float, qty: int):
        self.location = location
        self.length = length
        self.height = height
        self.qty = qty
        self.area = length * height / 1_000_000  # Convert mm to sqm

class StockSheet:
    def __init__(self, length: float, width: float):
        self.length = length
        self.width = width
        self.total_area = length * width / 1_000_000  # Convert mm to sqm

class HybridGlassCuttingOptimizer:
    def __init__(self, stock_sheets: List[StockSheet], glass_pieces: List[GlassPiece]):
        self.stock_sheets = stock_sheets
        self.glass_pieces = glass_pieces
        
    def optimize(self):
        # Sort pieces by area in descending order
        sorted_pieces = sorted(
            [piece for piece in self.glass_pieces for _ in range(piece.qty)],
            key=lambda p: p.length * p.height, 
            reverse=True
        )
        
        # Track utilization
        sheet_utilization = []
        remaining_pieces = sorted_pieces.copy()
        
        for stock_sheet in self.stock_sheets:
            sheet_cuts = self._optimize_single_sheet(stock_sheet, remaining_pieces)
            
            if sheet_cuts:
                sheet_util = sum(cut.length * cut.height for cut in sheet_cuts) / stock_sheet.total_area
                sheet_utilization.append({
                    'sheet_size': f"{stock_sheet.length}x{stock_sheet.width}",
                    'utilized_pieces': len(sheet_cuts),
                    'utilization_percentage': sheet_util * 100
                })
                
                # Remove used pieces
                for cut in sheet_cuts:
                    remaining_pieces.remove(cut)
        
        return {
            'sheet_utilization': sheet_utilization,
            'remaining_pieces': len(remaining_pieces)
        }
    
    def _optimize_single_sheet(self, stock_sheet, pieces):
        sheet_cuts = []
        remaining_pieces = pieces.copy()
        
        while remaining_pieces:
            best_piece = None
            best_efficiency = 0
            
            for piece in remaining_pieces:
                # Check if piece can fit in the stock sheet
                if (piece.length <= stock_sheet.length and piece.height <= stock_sheet.width) or \
                   (piece.height <= stock_sheet.length and piece.length <= stock_sheet.width):
                    
                    # Calculate potential efficiency
                    piece_area = piece.length * piece.height
                    sheet_area = stock_sheet.length * stock_sheet.width
                    efficiency = piece_area / sheet_area
                    
                    if efficiency > best_efficiency:
                        best_piece = piece
                        best_efficiency = efficiency
            
            if best_piece:
                sheet_cuts.append(best_piece)
                remaining_pieces.remove(best_piece)
            else:
                break
        
        return sheet_cuts



def main():
    # Read input data
    glass_data = pd.read_csv('data/glass_data.csv')
    stock_data = pd.read_csv('data/glass_sheet_size.csv')

    # Prepare glass pieces
    glass_pieces = [
        GlassPiece(
            location=row['location'], 
            length=row['glass_length'], 
            height=row['glass_height'], 
            qty=row['glass_qty']
        ) for _, row in glass_data.iterrows()
    ]
    
    # Prepare stock sheets
    stock_sheets = [
        StockSheet(length=row['length'], width=row['width']) 
        for _, row in stock_data.iterrows() 
        for _ in range(row['qty'])
    ]
    
    # Create optimizer
    optimizer = HybridGlassCuttingOptimizer(stock_sheets, glass_pieces)
    
    # Run optimization
    results = optimizer.optimize()
    
    # Print results
    print("Sheet Utilization Results:")
    for sheet in results['sheet_utilization']:
        print(f"Sheet Size: {sheet['sheet_size']}")
        print(f"Utilized Pieces: {sheet['utilized_pieces']}")
        print(f"Utilization Percentage: {sheet['utilization_percentage']:.2f}%\n")
    
    print(f"Remaining Uncut Pieces: {results['remaining_pieces']}")

if __name__ == "__main__":
    main()

Sheet Utilization Results:
Sheet Size: 3300x2438
Utilized Pieces: 260
Utilization Percentage: 5607447373.65%

Remaining Uncut Pieces: 0


In [4]:
import pandas as pd
import numpy as np
from typing import List, Tuple
import matplotlib.pyplot as plt
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch

class GlassPiece:
    def __init__(self, location: str, length: float, height: float, qty: int):
        self.location = location
        self.length = length
        self.height = height
        self.qty = qty
        self.area = length * height / 1_000_000  # Convert mm to sqm

class StockSheet:
    def __init__(self, length: float, width: float):
        self.length = length
        self.width = width
        self.total_area = length * width / 1_000_000  # Convert mm to sqm

class HybridGlassCuttingOptimizer:
    def __init__(self, stock_sheets: List[StockSheet], glass_pieces: List[GlassPiece]):
        self.stock_sheets = stock_sheets
        self.glass_pieces = glass_pieces
        
    def optimize(self):
        # Sort pieces by area in descending order
        sorted_pieces = sorted(
            [piece for piece in self.glass_pieces for _ in range(piece.qty)],
            key=lambda p: p.length * p.height, 
            reverse=True
        )
        
        # Track utilization
        used_sheets = []
        remaining_pieces = sorted_pieces.copy()
        total_part_area = sum(p.length * p.height for p in sorted_pieces)
        
        for stock_sheet in self.stock_sheets:
            sheet_cuts = self._optimize_single_sheet(stock_sheet, remaining_pieces)
            
            if sheet_cuts:
                used_sheets.append(stock_sheet)
                
                # Remove used pieces
                for cut in sheet_cuts:
                    remaining_pieces.remove(cut)
        
        return {
            'used_sheets': used_sheets,
            'total_part_area': total_part_area,
            'remaining_pieces': remaining_pieces
        }
    
    def _optimize_single_sheet(self, stock_sheet, pieces):
        sheet_cuts = []
        remaining_pieces = pieces.copy()
        
        while remaining_pieces:
            best_piece = None
            best_efficiency = 0
            
            for piece in remaining_pieces:
                # Check if piece can fit in the stock sheet
                if (piece.length <= stock_sheet.length and piece.height <= stock_sheet.width) or \
                   (piece.height <= stock_sheet.length and piece.length <= stock_sheet.width):
                    
                    # Calculate potential efficiency
                    piece_area = piece.length * piece.height
                    sheet_area = stock_sheet.length * stock_sheet.width
                    efficiency = piece_area / sheet_area
                    
                    if efficiency > best_efficiency:
                        best_piece = piece
                        best_efficiency = efficiency
            
            if best_piece:
                sheet_cuts.append(best_piece)
                remaining_pieces.remove(best_piece)
            else:
                break
        
        return sheet_cuts

def print_results(sheets: List[StockSheet], total_part_area: float):
    total_sheet_area = sum(sheet.total_area for sheet in sheets)
    used_area_percentage = (total_part_area / total_sheet_area) * 100
    wastage_percentage = 100 - used_area_percentage
    print(f"\nTotal sheets used: {len(sheets)}")
    print(f"Total part area: {total_part_area / 1_000_000:.2f} sq meters")
    print(f"Total sheet area used: {total_sheet_area:.2f} sq meters")
    print(f"Used area percentage: {used_area_percentage:.2f}%")
    print(f"Wastage percentage: {wastage_percentage:.2f}%")

class CuttingVisualization:
    def __init__(self, optimization_results):
        self.results = optimization_results
    
    def create_sheet_visualization_pdf(self, filename='cutting_optimization.pdf'):
        c = canvas.Canvas(filename, pagesize=letter)
        width, height = letter
        
        for idx, sheet in enumerate(self.results['sheet_utilization'], 1):
            # Create page for each sheet
            c.drawString(inch, height - inch, 
                         f"Sheet {idx}: {sheet['sheet_size']} mm")
            
            # Add utilization details
            c.drawString(inch, height - 1.5*inch, 
                         f"Utilization: {sheet['utilization_percentage']:.2f}%")
            
            # Next page
            c.showPage()
        
        c.save()
        print(f"Visualization PDF saved: {filename}")
    
    def plot_sheet_utilization(self):
        plt.figure(figsize=(10, 6))
        sheets = [sheet['sheet_size'] for sheet in self.results['sheet_utilization']]
        utilizations = [sheet['utilization_percentage'] for sheet in self.results['sheet_utilization']]
        
        plt.bar(sheets, utilizations)
        plt.title('Glass Sheet Utilization')
        plt.xlabel('Sheet Size')
        plt.ylabel('Utilization Percentage')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('sheet_utilization.png')
        plt.close()

# Integrate with main optimization script
def visualize_optimization(optimization_results):
    viz = CuttingVisualization(optimization_results)
    viz.create_sheet_visualization_pdf()
    viz.plot_sheet_utilization()

# Call in main script after optimization
# visualize_optimization(results)

def main():
    # Read input data
    glass_data = pd.read_csv('data/glass_data.csv')
    stock_data = pd.read_csv('data/glass_sheet_size.csv')
    
    # Prepare glass pieces
    glass_pieces = [
        GlassPiece(
            location=row['location'], 
            length=row['glass_length'], 
            height=row['glass_height'], 
            qty=row['glass_qty']
        ) for _, row in glass_data.iterrows()
    ]
    
    # Prepare stock sheets
    stock_sheets = [
        StockSheet(length=row['length'], width=row['width']) 
        for _, row in stock_data.iterrows() 
        for _ in range(row['qty'])
    ]
    
    # Create optimizer
    optimizer = HybridGlassCuttingOptimizer(stock_sheets, glass_pieces)
    
    # Run optimization
    results = optimizer.optimize()
    
    # Print results
    print_results(results['used_sheets'], 
                  sum(piece.length * piece.height for piece in glass_pieces))
    
    print(f"\nRemaining Uncut Pieces: {len(results['remaining_pieces'])}")

if __name__ == "__main__":
    main()


Total sheets used: 1
Total part area: 23.13 sq meters
Total sheet area used: 8.05 sq meters
Used area percentage: 287539935.86%
Wastage percentage: -287539835.86%

Remaining Uncut Pieces: 0


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch

class CuttingVisualization:
    def __init__(self, optimization_results):
        self.results = optimization_results
    
    def create_sheet_visualization_pdf(self, filename='cutting_optimization.pdf'):
        c = canvas.Canvas(filename, pagesize=letter)
        width, height = letter
        
        for idx, sheet in enumerate(self.results['sheet_utilization'], 1):
            # Create page for each sheet
            c.drawString(inch, height - inch, 
                         f"Sheet {idx}: {sheet['sheet_size']} mm")
            
            # Add utilization details
            c.drawString(inch, height - 1.5*inch, 
                         f"Utilization: {sheet['utilization_percentage']:.2f}%")
            
            # Next page
            c.showPage()
        
        c.save()
        print(f"Visualization PDF saved: {filename}")
    
    def plot_sheet_utilization(self):
        plt.figure(figsize=(10, 6))
        sheets = [sheet['sheet_size'] for sheet in self.results['sheet_utilization']]
        utilizations = [sheet['utilization_percentage'] for sheet in self.results['sheet_utilization']]
        
        plt.bar(sheets, utilizations)
        plt.title('Glass Sheet Utilization')
        plt.xlabel('Sheet Size')
        plt.ylabel('Utilization Percentage')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('sheet_utilization.png')
        plt.close()

# Integrate with main optimization script
def visualize_optimization(optimization_results):
    viz = CuttingVisualization(optimization_results)
    viz.create_sheet_visualization_pdf()
    viz.plot_sheet_utilization()

# Call in main script after optimization
visualize_optimization(results)

NameError: name 'results' is not defined